#### A simple model of the hardware: a chain of $n$ simple repeaters

    r[1] ---- r[2] ---- r[3] ---- ... ---- r[n]
    
A simple repeater: a repeater that can support 2 links that both have the same lifetime; and has ideal gates (applying gates on the qubits does not affect lifetimes).

In [0]:
from threading import Timer
import random
from IPython.display import clear_output


# Helper function: flips an unfair coin.
def flip_unfair_coin(p):
    return 'H' if random.random() < p else 'T'
    
    
class simple_repeater(object):
    # Constructor.
    def __init__(self, id, link_lifetime = 1, link_creation_time = 0.005, parent_chain = None):
        # The repeater id.
        self.id = id
        # The lifetime of the links, specifically the time in which right link expires
        self.link_lifetime = link_lifetime
        # The time needed to establish a link.
        self.link_creation_time = link_creation_time
        # The probability that a link is create successfully.
        self.link_creation_prob = 1
        # Repeaters/nodes to the right and to the left.
        # They are simple_repeater objects.
        # None means the link is not alive.
        self.rightNode = None
        self.leftNode  = None
        # Timers to keep track of the alive time of each link.
        self.rightLinkTimer = None
        self.leftLinkTimer = None
        self.rightLinkTimerStart=None
        # Repeater chain in which the repeater is embedded.
        self.parent_chain = parent_chain
        # The repeater's queued tasks.
#         self.task_queue = []

    # Link repeater to another one sitting somewhere on its right.
    def create_right_link_helper(self, other,expire_time=None):
        self.rightNode = other
        other.leftNode = self
        # Start expiry timer.
        if expire_time is not None:
          self.rightLinkTimer = Timer(expire_time, self.right_link_expires)
          self.link_lifetime=expire_time
        else:
          self.rightLinkTimer = Timer(self.link_lifetime, self.right_link_expires)
        self.rightLinkTimerStart=time.time()
        self.rightLinkTimer.start()
        if self.parent_chain.GUI_enabled:
            self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        else:
            print("link created between " + self.id + " and " + self.rightNode.id)
            # Check for success.
            if self.parent_chain.success():
               print("Success!") 
            
    def create_right_link(self, other):
        # Creating a link is probabilistic.
        if flip_unfair_coin(self.link_creation_prob) == 'H':
            # And takes time.
            Timer(self.link_creation_time, self.create_right_link_helper, [other]).start()
    
#     # Link repeater to another one sitting somewhere on its left.
#     def create_left_link_helper(self, other):
#         self.leftNode = other
#         other.rightNode = self
#         self.leftLinkTimer = Timer(self.link_lifetime, self.left_link_expires)
#         self.leftLinkTimer.start()
#         if self.parent_chain.GUI is not None:
#             self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            
#     def create_left_link(self, other):
#         if flip_unfair_coin(self.link_creation_prob) == 'H':
#             Timer(self.link_creation_time, self.create_left_link_helper, [other]).start()
        
    # Right link dies.
    def right_link_expires(self):
        if self.rightNode is not None:
            self.rightNode.leftNode = None
            self.rightNode = None
            self.rightLinkTimer = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        
    # Left link dies.
    def left_link_expires(self):
        if self.leftNode is not None:
            self.leftNode.rightNode = None
            self.leftNode = None
            self.leftLinkTimer = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())

    # Perform entanglement swap.
    def swap(self):
#         print("Swapping in repeater", self.id)
        if self.leftNode is not None and self.rightNode is not None:
            #time in which self.leftNode--self will expire, ie the qubits will decohere
            left_qb_expiry=self.leftNode.link_lifetime-(time.time()-self.leftNode.rightLinkTimerStart)
            #time in which self--self.rightNode will expire, ie the qubits will decohere
            right_qb_expiry=self.link_lifetime-(time.time()-self.rightLinkTimerStart)
            #the connection self.leftNode--self.rightNode will expire in the min of the above times
            connection_expiry=min(left_qb_expiry,right_qb_expiry)
            self.leftNode.create_right_link_helper(self.rightNode,connection_expiry)
            # Change link states to not alive.
            self.rightNode = None
            self.leftNode  = None
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            
     

class simple_repeater_chain(object):
    def __init__(self, length = 5, link_lifetime = 1, link_creation_time = 0.005,
                 GUI_enabled = False, repeater_type=simple_repeater):
        self.length = length
        self.link_lifetime = link_lifetime
        self.link_creation_time = link_creation_time
        self.link_creation_prob = 1
        # Initialize the repeaters.
        self.repeaters = []
        for i in range(self.length):
            new_repeater = repeater_type(str(i), link_lifetime, link_creation_time, self)
            self.repeaters.append(new_repeater)
        # Initialize the GUI.
        self.GUI_enabled = GUI_enabled
        if self.GUI_enabled:
            self.GUI = GUI()
#             self.GUI.draw_state(self.pack_snapshot())
            
    def success(self):
        if self.repeaters[0].rightNode is None:
            return False
        return self.repeaters[0].rightNode.id == self.repeaters[-1].id

    # Says which link each link is connected to.
    def pack_snapshot(self):
        snapshot = []  
        for repeater in self.repeaters:
            if repeater.rightNode is not None:
                snapshot.append(repeater.rightNode.id)
            else:
                # -1 means it's not linked to any repeaters.
                snapshot.append(-1)
#         print(snapshot)
        return snapshot

    

# GUI (not finished yet)
class GUI(object):
    def __init__(self):
        self.request_queue = []
        self.busy = False
        
    def draw_state(self, state):
        t1 = ""
        t2 = ""
        s = ""
        for i in range(len(state)):
#             s = s + "r" + "[" + str(i+1) + "]"
            s = s + "r"
            if i < len(state) - 1:
                if int(state[i]) == i + 1:
                    s += " ---- "
                    t1 += "  " + "    " + " "
                    t2 += "  " + "    " + " "
                else:
                    s += "      "
                    if state[i] != -1:
                        t1 += " /" + (int(state[i])-i-1) * ("    " + "   ") + "    " + "\\"
                        t2 += "  " + (int(state[i])-i-1) * ("----" + "---") + "----" + " "
                    else: 
                        t1 += "  " + "    " + " "
                        t2 += "  " + "    " + " "
        if int(state[0]) == len(state)-1:
            s += "       Success!"
        clear_output(wait = True)
        print(t2)
        print(t1)
        print(s)
        
    def update_gui(self, new_state):
        self.request_queue.append(new_state)
        if self.busy == False:
            self.get_busy()
        
    def get_busy(self):
        self.busy = True
        while len(self.request_queue) > 0:
            new_state = self.request_queue.pop(0)
            self.draw_state(new_state)
        self.busy = False 

In [0]:
class repeater_v2(simple_repeater):
  def __init__(self, id, link_lifetime = 1, link_creation_time = 0.005, parent_chain = None):
    
    super().__init__(id, link_lifetime, link_creation_time, 
                      parent_chain)

  def create_right_link_helper(self, other,expire_time=None):
      # Creating a link is probabilistic. returns True if successful.
      should_create = flip_unfair_coin(self.link_creation_prob) == 'H'
      if should_create:
        super().create_right_link_helper(other,expire_time)
        return True
      else:
        return False 
  
  def create_right_link(self, other, retry=True, retry_count=3):
      retry_count = retry_count if retry else 1
      for _ in range(retry_count):
          # Creating link takes time.
          if self.rightNode is None:
            success = self.create_right_link_helper(other)
            if success:
              return
            time.sleep(self.link_creation_time)
          self.parent_chain.indicate_link_failiure([self, other])
    
  # Right link dies.
  def right_link_expires(self):
    if self.rightNode is not None:
      other = self.rightNode
      super().right_link_expires()
      # self.parent_chain.indicate_link_failiure([self, other]) // yet to decide if the link expires, will it be same as link failiure?
        
  # Left link dies.
  def left_link_expires(self):
    if self.leftNode is not None:
      other = self.leftNode
      super().left_link_expires()
        # self.parent_chain.indicate_link_failiure([self, other]) 


class repeater_v2_chain(simple_repeater_chain):
  STATE_INIT, STATE_ACTIVE, STATE_BROKEN = 'INIT', 'ACTIVE', 'BROKE' 
  
  def __init__(self, length = 5, link_lifetime = 1, link_creation_time = 0.005, GUI_enabled = False):
    super().__init__(length, link_lifetime, link_creation_time, GUI_enabled, repeater_type=repeater_v2)

    # self.failed_links = [] #In future version of this protocol we can make use of the failed links to re-route.
    self.state = self.STATE_INIT


  def indicate_link_failiure(self, repeaters):
      """Repeaters should call this function to indicate link creation failiure.
       We can add broadcast function in future to inform all nodes that they can free resources since complete link was not established. 
      """
      # self.failed_links.append(repeaters)
      self.state = self.STATE_BROKEN
      print(f'Link creation failiure at: {repeaters[0].id}-{repeaters[1].id}.')

  def make_chain_active(self):
    if self.state != self.STATE_BROKEN:
      self.state = self.STATE_ACTIVE

  def is_active(self):
    return self.state == self.STATE_ACTIVE

#### Our goal is to end up with the first and last repeaters --- they are really nodes --- in the chain entangled.

#### Here's a possible protocol

In [0]:
import time

# Try once to establish all links, starting from the leftmost repeater. Then swap them one by one.
def protocol1(simple_repeater_chain):
    # Create links
    for i in range(len(simple_repeater_chain.repeaters)-1):
        this_repeater = simple_repeater_chain.repeaters[i]
        next_repeater = simple_repeater_chain.repeaters[i+1]
        this_repeater.create_right_link(next_repeater)
        # Wait a bit
        time.sleep(this_repeater.link_creation_time)
    # Perform swap.
    for repeater in simple_repeater_chain.repeaters:
        repeater.swap()
        # Wait a bit
        time.sleep(repeater.link_creation_time)

#### Let's test it
Play with the parameters and see what happens!

In [0]:
chain = simple_repeater_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5, 
                              GUI_enabled = True)


protocol1(chain)

                                                 
                                                 
r      r      r      r      r      r      r      r


#### Protocol 2
Here is an alternate protocol that attempts to establish connection between nodes 0 and i+1 as soon as connection between 0 and i is established. This ensures that the first established link does not expire by the time connection between the last two nodes is established

In [0]:
def protocol2(simple_repeater_chain):
  for i in range(len(simple_repeater_chain.repeaters)-1):
    this_repeater = simple_repeater_chain.repeaters[i]
    next_repeater = simple_repeater_chain.repeaters[i+1]
    # print("creating right link from i=",i)
    this_repeater.create_right_link(next_repeater)
    # Wait a bit
    time.sleep(this_repeater.link_creation_time)
    #perform swap if it is not the first node
    if(i!=0):
      # print("doing swap at i=",i)
      this_repeater.swap()
      # Wait a bit
      time.sleep(this_repeater.link_creation_time)

#### Testing protocol2

In [0]:
chain = simple_repeater_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5,
                              GUI_enabled = True)


protocol2(chain)

                                                 
                                                 
r      r      r      r      r      r      r ---- r


#### Protocol 1.1

This protocol improves upon Protocol 1 by:
* Only allowing swapping if the complete chain is ready.
* Inform all the repeaters at once to create link. 

##### Protocol definition
* Inform all the nodes at once which node they need to link with (can be done classically) and then wait for the time it would take maximum possible retries to complete.

* Incase link creation fails, repeaters retry link creation. If all attempts for link creation fail, repeaters can inform parent chain/broadcast that the link creation  failed or link expired. If Parent chain receives information on link creation failiure, link is considered in broken state.

* If no error was received until the (retries allowed * link_creation_time) seconds, the link is made active.

* Start `swap()` operations  and keep swapping if the chain is in Active State.

(PS. please use this protocol with GUI_enabled=False, does not work well with GUI)
 

In [0]:
def protocol1_1(repeater_chain):
    # Create links
    retries = 3
    for i in range(len(repeater_chain.repeaters)-1):
        # Inform all the repeaters simaltaneously whom they'll be linking up with.
        this_repeater = repeater_chain.repeaters[i]
        next_repeater = repeater_chain.repeaters[i+1]
        this_repeater.create_right_link(next_repeater, retry_count=retries)

    # Wait for a repeaters to indicate to `Parent Chain` if they failed in creating the link.
    time.sleep(repeater_chain.link_creation_time * retries)
    repeater_chain.make_chain_active() # If no failiures, make active.
    
    # Perform swap.
    for repeater in repeater_chain.repeaters:
        if repeater_chain.is_active():
          repeater.swap()
          # Wait a bit
          time.sleep(repeater.link_creation_time)

In [0]:
chain = repeater_v2_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5, GUI_enabled = False, )
protocol1_1(chain)

link created between 0 and 1
link created between 1 and 2
link created between 2 and 3
link created between 3 and 4
link created between 4 and 5
link created between 5 and 6
link created between 6 and 7
link created between 0 and 2
link created between 0 and 3
link created between 0 and 4
link created between 0 and 5
link created between 0 and 6
link created between 0 and 7
Success!


#### Maintaining multiple parallel entanglements

For creating an entanglement between repeaters at 1 and n, we first need to creat entanglement between nodes at 1 and (n-1) and those at (n-1) and n. Then we can easily swap at the (n-1)th node to achieve the result. Consider that entanglement between 1 and (n-1) has been created. Now, we might fail to connect 1 and n if the link between 1 and (n-1) expires by the time (n-1) and n are connected and (n-1) gets some classically communicated signal to do the entanglement swapping. 

To owercome this, we try to maintain w entangled pairs between 1 and (n-1) at all points so that by the time one of the entangled pairs expires, we have created a new one. This way, when the node at (n-1) gets the signal to swap, it will necessarily have a shared pair with node 1. We can do the same for (n-1) and n as well.

Let's assume that it takes time c for 1 and (n-1) to share an entangled pair, one of the qubits of the pair decoheres e time after the entanglement is achieved, we start the process of creating new entangled pairs in every t time and we are maintaining w entangled qubits at nodes 1 and (n-1). So, we start creating the first pair at time t=0 and the (w+1)th pair at time wt. The first pair expires at time (e+c) and the (w+1)th pair is created at time wt+c. In order for the total entangled pairs to remain constant we thus have,

$$wt=e$$

We only need w=1 for the above mentioned protocol, but I take w=2 to be on the safe side in the protocol below.

Please use the protocol without GUI for now. Also note that there may be more output being printed after reporting that the link was created successfully. These are generated due to threads which had already started when link creation was completed and should be ignored.

In [0]:
import threading
import time
class multiple_connection_repeater(simple_repeater):
    # Constructor.
    def __init__(self, id, link_lifetime = 1, link_creation_time = 0.005, parent_chain = None):
        #initialise parent class object
        super().__init__(id,link_lifetime,link_creation_time,parent_chain)

        #dict from nodes self is connected to to a dict containing information about each connection. 
        self.connections={}

        self.pendingTimers={}

    # Link repeater to another one sitting somewhere on its right.
    def create_right_link_helper(self, other,expire_time=None):
        # print("create right link helper called for ",self.id," and ",other.id," by thread id: ",threading.get_ident())
        if expire_time is None:
            expire_time=self.link_lifetime
        if other in self.connections:
            conn_id=self.connections[other]["used_id"]+1
        else:
            self.connections[other]={}
            conn_id=0
        if not self in other.connections:
            other.connections[self]={}
        if "conn_list" not in self.connections[other]:
            self.connections[other]["conn_list"]={}
        if "conn_list" not in other.connections[self]:
            other.connections[self]["conn_list"]={}
        conn_dict={}
        conn_dict["timer"]=Timer(expire_time, self.right_link_expires,[other,conn_id])
        conn_dict["life"]=expire_time
        conn_dict["start_time"]=time.time()
        self.connections[other]["conn_list"][conn_id]=conn_dict
        self.connections[other]["used_id"]=conn_id
        other.connections[self]["used_id"]=conn_id
        other.connections[self]["conn_list"][conn_id]=conn_dict

        conn_dict["timer"].start()
        # print("link expiry timer of id: ",conn_dict["timer"].ident," has been started with timeout: ",conn_dict["life"])
        print("link between ",self.id," and ",other.id," will expire in ",conn_dict["life"]," seconds")
        if self.parent_chain.GUI_enabled:
            self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        else:
            print("link created between " , self.id , " and " , other.id," with id: ",conn_id)
            # Check for success.
            if self.parent_chain.success():
               print("Success!")
            
    def create_right_link(self, other):
        # Creating a link is probabilistic.
        if flip_unfair_coin(self.link_creation_prob) == 'H':
            # And takes time.
            # t=Timer(self.link_creation_time, self.create_right_link_helper, [other])
            # t.start()
            # print("link creation timer of id: ",t.ident," has been started with timeout: ",self.link_creation_time)
            print("waiting for link creation between ",self.id," and ",other.id)
            time.sleep(self.link_creation_time)
            self.create_right_link_helper(other)
    
#     # Link repeater to another one sitting somewhere on its left.
#     def create_left_link_helper(self, other):
#         self.leftNode = other
#         other.rightNode = self
#         self.leftLinkTimer = Timer(self.link_lifetime, self.left_link_expires)
#         self.leftLinkTimer.start()
#         if self.parent_chain.GUI is not None:
#             self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            
#     def create_left_link(self, other):
#         if flip_unfair_coin(self.link_creation_prob) == 'H':
#             Timer(self.link_creation_time, self.create_left_link_helper, [other]).start()
        
    # Right link dies.
    def right_link_expires(self,other,conn_id,used=False):
        if other in self.connections and "conn_list" in self.connections[other] and conn_id in self.connections[other]["conn_list"]:
            # print("cancelling the timer of id: ",self.connections[other]["conn_list"][conn_id]["timer"].ident)
            self.connections[other]["conn_list"][conn_id]["timer"].cancel()
            del self.connections[other]["conn_list"][conn_id]
        if self in other.connections and "conn_list" in other.connections[self] and conn_id in other.connections[self]["conn_list"]:
            # print("cancelling the timer of id: ",other.connections[self]["conn_list"][conn_id]["timer"].ident)
            other.connections[self]["conn_list"][conn_id]["timer"].cancel()
            del other.connections[self]["conn_list"][conn_id]
        if self.parent_chain.GUI_enabled:
            self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
        elif not used:
            print("connection between ",self.id," and ",other.id," of id: ",conn_id," has expired")
        else:
            print("connection between ",self.id," and ",other.id," of id: ",conn_id," has been used")

    # Perform entanglement swap.
    def swap(self,leftNode,rightNode):
#         print("Swapping in repeater", self.id)
        if leftNode in self.connections and "conn_list" in self.connections[leftNode] and not(self.connections[leftNode]["conn_list"]=={}) and rightNode in self.connections and "conn_list" in self.connections[rightNode] and not(self.connections[rightNode]["conn_list"]=={}):
            left_id=None
            max_time=0
            for i,conn in self.connections[leftNode]["conn_list"].items():
                if (conn["start_time"]+conn["life"]-time.time())>max_time:
                    max_time=(conn["start_time"]+conn["life"]-time.time())
                    left_id=i

            right_id=None
            max_time=0
            for i,conn in self.connections[rightNode]["conn_list"].items():
                if (conn["start_time"]+conn["life"]-time.time())>max_time:
                    max_time=(conn["start_time"]+conn["life"]-time.time())
                    right_id=i
            #time in which self.leftNode--self will expire, ie the qubits will decohere
            left_qb_expiry=self.connections[leftNode]["conn_list"][left_id]["life"]-(time.time()-self.connections[leftNode]["conn_list"][left_id]["start_time"])
            #time in which self.leftNode--self will expire, ie the qubits will decohere
            right_qb_expiry=self.connections[rightNode]["conn_list"][right_id]["life"]-(time.time()-self.connections[rightNode]["conn_list"][right_id]["start_time"])
            connection_expiry=min(left_qb_expiry,right_qb_expiry)
            leftNode.create_right_link_helper(rightNode,connection_expiry)
            leftNode.right_link_expires(self,left_id,True)
            self.right_link_expires(rightNode,right_id,True)
            if self.parent_chain.GUI_enabled:
                self.parent_chain.GUI.update_gui(self.parent_chain.pack_snapshot())
            else:
                print("Used the connection ",left_id," between ",leftNode.id," and ",self.id," and ",right_id," between ",self.id," and ",rightNode.id," for swapping")
            return connection_expiry
        else:
            print("Swap between ",leftNode.id," and ",self.id," and ",rightNode.id," failed")
            return 0

class mult_repeater_chain(simple_repeater_chain):
    def __init__(self, length = 5, link_lifetime = 1, link_creation_time = 0.005,
                 GUI_enabled = False, repeater_type=multiple_connection_repeater):
        super().__init__(length,link_lifetime,link_creation_time,GUI_enabled,repeater_type)
            
    def success(self):
        if self.repeaters[self.length-1] in self.repeaters[0].connections and "conn_list" in self.repeaters[0].connections[self.repeaters[self.length-1]] and not(self.repeaters[0].connections[self.repeaters[self.length-1]]["conn_list"]=={}):
            # print(self.repeaters[0].connections[self.repeaters[self.length-1]]["conn_list"])
            for i,conn in self.repeaters[0].connections[self.repeaters[self.length-1]]["conn_list"].items():
                print("#########\n\n\n\n\nconnection made with id: ",i,"\n\n\n\n\n#########")
                for i in range(0,len(self.repeaters)):
                    for node,timers in self.repeaters[i].pendingTimers.items():
                        for timer in timers:
                            # print("cancelling the timer of id: ",timer.ident," all is done")
                            timer.cancel()
                    for node,conn in self.repeaters[i].connections.items():
                        for id,conn_dict in conn.get("conn_list",{}).items():
                            # print("cancelling the timer of id: ",conn_dict["timer"].ident," all is done")
                            conn_dict["timer"].cancel()
                exit()
                return True
        return False

    # Says which link each link is connected to.
    def pack_snapshot(self):
        pass

In [0]:
# link between left and right will not break after this function returns
def protocol_mult_conn_helper(repeater_chain,left_index,right_index,expiry_time=None,topmost=False):
    # print("protocol helper called for ",left_index,", ",right_index," with timer of id: ",threading.get_ident())
    print("protocol helper called for ",left_index,", ",right_index)
    # expiry_time is None if we are not already aware about the time after which the link fails after connection is made
    if expiry_time is not None and not topmost:#no need to call again if this is the first call
        if repeater_chain.repeaters[right_index] not in repeater_chain.repeaters[left_index].pendingTimers:
            repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]]=[]
        
        # this function should be called after every expiry_time/2 time , create and start the timer
        temp_timer=Timer(expiry_time/2,protocol_mult_conn_helper,[repeater_chain,left_index,right_index,expiry_time])
        repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]].append(temp_timer)
        repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]][-1].start()
        # print("protocol helper timer of id: ",temp_timer.ident," has been started with timeout: ",expiry_time/2)
    start_time=time.time()

    if not (right_index==left_index+1):
        # connect left and right-1 and ensure their connection remains intact
        protocol_mult_conn_helper(repeater_chain,left_index,right_index-1)
    
    # calculate expiry time in case we do not know it already
    # if there is only one connection to make, then it will expire in link_lifetime period
    calc_expiry_time=repeater_chain.repeaters[left_index].link_lifetime

    # create the link between the last two nodes
    repeater_chain.repeaters[right_index-1].create_right_link(repeater_chain.repeaters[right_index])
    #time.sleep(repeater_chain.repeaters[right_index-1].link_creation_time)
    if not (right_index==left_index+1):
        # link between 1 and (n-1) and (n-1) and n is created. expiry time is as returned by swap
        calc_expiry_time=repeater_chain.repeaters[right_index-1].swap(repeater_chain.repeaters[left_index],repeater_chain.repeaters[right_index])

        # once a connection between 1 and n has been made, calls for 1 and (n-1) do not need to be made
        for timer in repeater_chain.repeaters[left_index].pendingTimers.get(repeater_chain.repeaters[right_index-1],[]):
            # print("cancelling the timer of id: ",timer.ident," swap done")
            timer.cancel()
        repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]]=[]

    if expiry_time is None and not topmost:#no need to call again if this is the first call
        # timer for calling the function again has not been set yet
        # this function should be called in the (calculated expiry time)/2 - (time elapsed in the current call) so that it is called at the correct moment
        t=calc_expiry_time/2-(time.time()-start_time)
        if repeater_chain.repeaters[right_index] not in repeater_chain.repeaters[left_index].pendingTimers:
            repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]]=[]
        temp_timer=Timer(t,protocol_mult_conn_helper,[repeater_chain,left_index,right_index,calc_expiry_time])
        repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]].append(temp_timer)
        repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]][-1].start()
        # print("protocol helper timer of id: ",temp_timer.ident," has been started with timeout: ",t)

def protocol_mult_conn(repeater_chain,left_index,right_index):
    protocol_mult_conn_helper(repeater_chain,left_index,right_index,topmost=True)
    # the connection is made now, remove all timers
    for i in range(left_index,right_index+1):
        for node,timers in repeater_chain.repeaters[i].pendingTimers.items():
            for timer in timers:
                # print("cancelling the timer of id: ",timer.ident," all is done")
                timer.cancel()
        for node,conn in repeater_chain.repeaters[i].connections.items():
            for id,conn_dict in conn.get("conn_list",{}).items():
                # print("cancelling the timer of id: ",conn_dict["timer"].ident," all is done")
                conn_dict["timer"].cancel()
    # if repeater_chain.repeaters[right_index] in repeater_chain.repeaters[left_index].pendingTimers.items():
    #     for node,timer in repeater_chain.repeaters[left_index].pendingTimers:
    #         print("cancelling the timer of id: ",timer.ident," all is done")
    #         timer.cancel()
    #     repeater_chain.repeaters[left_index].pendingTimers[repeater_chain.repeaters[right_index]]=[]
    # for i,d in repeater_chain.repeaters[left_index].connections[repeater_chain.repeaters[right_index]]["conn_list"].items():
    #     print("cancelling link expiry timer of id: ",d["timer"].ident)
    #     d["timer"].cancel()

In [6]:
chain = mult_repeater_chain(length = 8, link_lifetime = 5, link_creation_time = 0.5, GUI_enabled = False)
protocol_mult_conn(chain,0,len(chain.repeaters)-1)

protocol helper called for  0 ,  7
protocol helper called for  0 ,  6
protocol helper called for  0 ,  5
protocol helper called for  0 ,  4
protocol helper called for  0 ,  3
protocol helper called for  0 ,  2
protocol helper called for  0 ,  1
waiting for link creation between  0  and  1
link between  0  and  1  will expire in  5  seconds
link created between  0  and  1  with id:  0
waiting for link creation between  1  and  2
link between  1  and  2  will expire in  5  seconds
link created between  1  and  2  with id:  0
link between  0  and  2  will expire in  4.49616551399231  seconds
link created between  0  and  2  with id:  0
connection between  0  and  1  of id:  0  has been used
connection between  1  and  2  of id:  0  has been used
Used the connection  0  between  0  and  1  and  0  between  1  and  2  for swapping
waiting for link creation between  2  and  3
link between  2  and  3  will expire in  5  seconds
link created between  2  and  3  with id:  0
link between  0  and